# Draft analysis 

---

**Analyse des Kohortenverlaufs im Loyalty-Programm**

---


## Setup

In [119]:
import pandas as pd
from IPython.display import HTML
import altair as alt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Introduction

Die Daten wurden im Rahmen des initialen Business Case für das Loyalty Program von Lidl mit Hilfe einer Consulting erhoben. Für das Projekt wurden die tatsächlichen Daten verfälscht.  

Der Datensatz zeigt den Verlauf einzelner Kohorten, die aus Neukunden eines Loyalty-Programms bestehen. Eine Kohorte umfasst alle Kunden, die innerhalb eines Kalendermonats dem Programm beitreten. Durch die eindeutige Zuordnung der Kunden-IDs ist es möglich, individuelles Verhalten über einen längeren Zeitraum hinweg zu verfolgen. Dies erlaubt eine tiefgehende Untersuchung des Kohortenverhaltens im Zeitverlauf. Die verwendeten Kennzahlen und Merkmale des Datensatzes werden im Data Dictionary näher erläutert. 

Die Motivation für dieses Projekt ist es die Retentionrate je Kohorte je Monat vorherzusagen. Je mehr Kunden sich identifizieren, desto transparenter ist das Einkaufsverhalten der Kunden. Innerhalb des Projekts werden nur Bestandskunden betrachtet. Der Zuwachs durch neue Kunden ist eine weitere Variable, die vorhergesagt werden muss und daher nicht Teil dieses Projekts. Es besteht die Hypothese, dass die Zeit, in der die Kohorte entstanden ist, einen großen Einfluss auf das Identifizierungsverhalten hat. Ebenfalls hat der Zeitverlauf an sich einen Einfluss auf das Identifizierungsverhalten einer Kohorte, da unterschiedliche Monate sich unterschiedlich auf das Einkaufsverhalten auswirken können (z.B. Weihnachtszeit vs. Sommermonate). Um den Einfluss von Marketingaktivitäten abzubilden, wird als zusätzlicher Prediktor ausgespielte Rabatte in dem Monat an die Kohorte in dem Datensatz inkludiert.

In [49]:
# Lade das Data Dictionary
with open('../references/styled_data_dictionary.html', 'r') as f:
    html = f.read()

# Zeige das Data Dictionary an
HTML(html)


,Name,Format,Type,Role,Description
0,monate_seit_einfuehrung_programm_kohorte,object,ordinal,Predictor,"Hier wird gezeigt, in welchem Zeitraum seit offizieller Einführung des Programms die Kohorte entstanden ist. Ist der Wert hier 0 so ist die Kohorte entstanden, in dem Monat, in dem auch das Programm eingeführt wurde. Negative Werte resultieren aus Testzeiträumen, die dem Modell extra vermittelt werden müssen oder ganz aus dem Datensatz entfernt werden müssen."
1,monat,object,ordinal,Predictor,"Der Monat sagt aus, in welchem Monat das Einkaufsverhalten einer Kohorte aufgenommen wurde."
2,monat_jahr,object,ordinal,Predictor,"Der Jahreswert aus der Spalte ""monat""."
3,monat_monat,object,ordinal,Predictor,"Der Monatswert aus der Spalte ""monat""."
4,monat_jahreszeit,object,nominal,Predictor,"Die Jahreszeit aus der Spalte ""monat""."
5,kohorte,object,ordinal,Predictor,"Der Monat, an dem die Kohorte entstanden ist."
6,kohorte_jahr,object,ordinal,Predictor,"Der Jahreswert aus der Spalte ""kohorte""."
7,kohorte_monat,object,ordinal,Predictor,"Der Monatswert aus der Spalte ""kohorte""."
8,kohorte_jahreszeit,object,nominal,Predictor,"Die Jahreszeit aus der Spalte ""kohorte""."
9,erster_monat_kohorte_fg,bool,nominal,Predictor,"Hier wird eine Flag gesetzt, wenn der Monat der erste einer Kohorte ist. Hier ist der Wert der Spalte ""monat"" und der Spalte ""kohorte"" also gleich. Die Flag könnte für das Modell wichtig sein, da der erste Monat einer Kohorte sehr auffällig im Einkaufsverhalten ist und für das normale Verhalten nicht repräsentativ ist."


## Data

## Import data

In [50]:
path_data = 'https://raw.githubusercontent.com/mm391-030401/project/refs/heads/main/data/processed/'
file_data= 'data_final.csv'

data = pd.read_csv(path_data + file_data, sep=',', encoding='utf-8')

# Obwohl diese Spalten bereits im Notebook "02_MM_convert_columns" zu Strings konvertiert wurden, wurden sie beim erneuten Einlesen wieder als Integer erkannt
# Allerdings ist es sinnvoll, diese Spalten nicht als Integer (diskret) zu betrachten 
# sondern als ordinale Variablen (Rechenoperationen sind nicht sinnig für die Zeitangaben)
to_str_cols = ['monate_seit_einfuehrung_programm_kohorte', 'monat', 'monat_jahr',
       'monat_monat', 'kohorte', 'kohorte_jahr', 'kohorte_monat', 'monate_seit_existenz_kohorte']

# Konvertierung der Spalten in string 
for col in to_str_cols: 
    data[col] = data[col].astype(str)

### Data structure

In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 703 entries, 0 to 702
Data columns (total 15 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   monate_seit_einfuehrung_programm_kohorte  703 non-null    object 
 1   monat                                     703 non-null    object 
 2   monat_jahr                                703 non-null    object 
 3   monat_monat                               703 non-null    object 
 4   monat_jahreszeit                          703 non-null    object 
 5   kohorte                                   703 non-null    object 
 6   kohorte_jahr                              703 non-null    object 
 7   kohorte_monat                             703 non-null    object 
 8   kohorte_jahreszeit                        703 non-null    object 
 9   erster_monat_kohorte_fg                   703 non-null    bool   
 10  monate_seit_existenz_kohorte          

In [52]:
data.head()

,monate_seit_einfuehrung_programm_kohorte,monat,monat_jahr,monat_monat,monat_jahreszeit,kohorte,kohorte_jahr,kohorte_monat,kohorte_jahreszeit,erster_monat_kohorte_fg,monate_seit_existenz_kohorte,kohortengroesse_indexiert,identifizierte_kunden_indexiert,rabatt_indexiert,retentionrate
0,-2,201408,2014,8,Sommer,201408,2014,8,Sommer,True,0,0.41,0.41,2.54,100.000000
1,-2,201409,2014,9,Herbst,201408,2014,8,Sommer,False,1,0.41,0.40,7.14,97.560976
2,-2,201410,2014,10,Herbst,201408,2014,8,Sommer,False,2,0.41,0.39,9.28,95.121951
3,-2,201411,2014,11,Herbst,201408,2014,8,Sommer,False,3,0.41,0.38,3.22,92.682927
4,-2,201412,2014,12,Winter,201408,2014,8,Sommer,False,4,0.41,0.38,7.15,92.682927


### Data corrections 
Diese wurden bereits im Notebook "01_MM_feature_engineering.ipynb" sowie "02_MM_convert_columns.ipynb" vorgenommen.  
- Aus des Spalten `monat` sowie `kohorte` wurden weitere Spalten eingeführt (`monat_jahr`, `monat_monat`, `monat_jahreszeit`, `kohorte_jahr`, `kohorte_monat`, `kohorte_jahreszeit`)
- Die Spalte `retentionrate` wurde als berechnete Spalte aus den Spalten `kohortengroesse_indexiert` und `identifizierte_kunden_indexiert` hinzugefügt.
- Die Spaltenbezeichnungen wurden transformiert, um die Arbeit leichter zu machen.
- Die Datenformate wurden angepasst.

### Variable lists

In [87]:
y_label = 'retentionrate'
list_numeric = data.select_dtypes(include = 'number').columns.to_list()
list_category = data.select_dtypes(include = ['object', 'bool']).columns.to_list()
x = ['kohorte_monat', 'kohorte_jahr', 'monat_monat', 'monat_jahr']

## Analysis

### Descriptive statistics

In [54]:
data[list_numeric].describe().T

,count,mean,std,min,25%,50%,75%,max
kohortengroesse_indexiert,703.0,47.926743,77.462511,0.410000,23.110000,24.850000,33.930000,375.13
identifizierte_kunden_indexiert,703.0,28.301565,53.452328,0.310000,10.890000,12.320000,21.215000,375.13
rabatt_indexiert,703.0,99.719090,257.969362,2.260000,21.500000,33.020000,61.045000,2570.59
retentionrate,703.0,55.442768,15.491696,35.752785,46.394922,49.102845,57.336771,100.00


In [55]:
data[list_category].describe().T

,count,unique,top,freq
monate_seit_einfuehrung_programm_kohorte,703,37,-2,37
monat,703,37,201708,37
monat_jahr,703,4,2016,282
monat_monat,703,12,8,76
monat_jahreszeit,703,4,Sommer,217
kohorte,703,37,201408,37
kohorte_jahr,703,4,2015,318
kohorte_monat,703,12,8,76
kohorte_jahreszeit,703,4,Herbst,207
erster_monat_kohorte_fg,703,2,False,666


- `kohortengroesse_indexiert` sowie `identifizierte_kunden_indexiert` zeigen eine hohe Diskrepanz zwischen dem 3. Quantil und dem Maximalwert. Es gibt also wenige Kohorten, die eine sehr hohe Anzahl an Kunden ausweisen. Je höher die anfängliche Kundenanzahl desto höher dann auch die Anzahl der möglichen Kunden, die sich jeden Monat identifizieren können
- Ebenfalls gibt es bestimmte Kohorten-Monats-Kombinationen, die einen hohen Rabatt erhalten haben. 
- Die maximale Retentionrate liegt bei 100. Im ersten Monat der Kohorte (dann, wenn die Kohorte entsteht) ist der Wert von Spalte `kohortengroesse_indexiert` und `identifizierte_kunden_indexiert` immer gleich. Hier liegt also die Retentionrate bei 100. Der starke Kundenrückgang nach dem ersten Monat einer Kohorte zeigt sich in den Werten von den Quantilen. Die Differenz zwischen 3. Quantil und Maximalwert ist ebenfalls hoch

### Exploratory data analysis

#### Numeric Data

In [56]:
alt.Chart(data).mark_circle().encode(
    x=alt.X(alt.repeat("column"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
            ),
    y=alt.Y(alt.repeat("row"), 
            type='quantitative',
            scale=alt.Scale(zero=False)
             )
).properties(
    width=150,
    height=150
).repeat(
    row=list_numeric,
    column=list_numeric
)

alt.RepeatChart(...)

Da Identifizierten Kunden und die Kohortengröße mit der Response Variable zusammenhängen, werden sie nicht als geeignet angesehen, trotz wahrscheinlich hoher Korrelation. Andernfalls wird das Modell falsch beeinflusst und wird keine zuverlässigen Output generieren. Der Rabatt scheint einen Einfluss zu haben und wird daher nochmal näher betrachtet.

In [57]:
alt.Chart(data).mark_circle().encode(
    x = alt.X('rabatt_indexiert'),
    y = alt.Y(y_label),
    tooltip = ['kohorte', 'monat']
).interactive()

alt.Chart(...)

Die rechten Ausreißer stammen ausschließlich aus der Kohorte 202410. Die Kohorte beansprucht den meisten Rabatt. Damit ein möglicher Zusammenhang nicht nur durch die oberen Ausreißer erscheint, wird sich der Scatterplot nochmal nur für diese Kohorte und einmal für alle ohne diese Kohorte angeschaut.

In [59]:
data_201410 = data[data['kohorte'] == '201410']

alt.Chart(data_201410).mark_circle().encode(
    x = alt.X('rabatt_indexiert'),
    y = alt.Y(y_label),
    tooltip = ['kohorte', 'monat']
).interactive()

alt.Chart(...)

In [60]:
data_wo_201410 = data[data['kohorte'] != '201410']

alt.Chart(data_wo_201410).mark_circle().encode(
    x = alt.X('rabatt_indexiert'),
    y = alt.Y(y_label),
    tooltip = ['kohorte', 'monat']
).interactive()

alt.Chart(...)

Nach Betrachtung der Daten wirkt es fast so, als ob jede Kohorte seinem eigenem Muster folgt. Daher wird einmal für jede Kohorte ein Scatterplot erstellt.

In [61]:
# Liste der einzigartigen Werte in der Spalte 'kohorte'
unique_kohorten = data['kohorte'].unique()

In [62]:
# Leere Liste, um die Charts zu speichern
charts = []

# Schleife über alle einzigartigen Werte in 'kohorte'
for kohorte in unique_kohorten:
    # Filtere den DataFrame nach dem aktuellen 'kohorte'-Wert
    df_filtered = data[data['kohorte'] == kohorte]
    
    # Erstelle das Diagramm
    chart = alt.Chart(df_filtered).mark_circle().encode(
        x=alt.X('rabatt_indexiert'),
        y=alt.Y(y_label),
        tooltip=['kohorte', 'monat']
    ).interactive()
    
    # Füge eine Überschrift hinzu
    chart = chart.properties(
        title=f'Kohorte {kohorte}'
    )
    
    # Füge das Diagramm der Liste hinzu
    charts.append(chart)

# Erstelle ein Rasterlayout aus den Diagrammen
grid_chart = alt.concat(*charts, columns=3) 

# Zeige das Raster an
grid_chart.display()



alt.ConcatChart(...)

Bei den meisten Kohorten scheint eine positive Korrelation zwischen dem Rabatt und der Retentionrate zu exisitieren. Dies wird nun mit einer Berechnung der Korrelation je Kohorte überprüft.

In [63]:
# Leere Liste, um die Ergebnisse zu speichern
korrelations_liste = []

# Schleife über alle einzigartigen Werte in 'kohorte'
for kohorte in unique_kohorten:
    # Filtere den DataFrame nach dem aktuellen 'kohorte'-Wert
    df_filtered = data[data['kohorte'] == kohorte]

    # Berechne die Korrelation (Es wird Spearman verwendet, da bereits in der deskriptven Statistik sowie in den Scatterplots deutlich wird, dass es Ausreißer gibt.)
    korrelation = df_filtered[['retentionrate', 'rabatt_indexiert']].corr(method='spearman').iloc[0, 1]
    
    # Füge die Ergebnisse zur Liste hinzu
    korrelations_liste.append({
        'Kohorte': kohorte,
        'Korrelation': korrelation
    })

# Erstelle einen DataFrame aus der Liste
korrelations_df = pd.DataFrame(korrelations_liste)

korrelations_df


,Kohorte,Korrelation
0,201408,0.075924
1,201409,0.186522
2,201410,0.645098
3,201411,0.639419
4,201412,0.687500
5,201501,0.416575
6,201502,0.296804
7,201503,0.222099
8,201504,0.122952
9,201505,0.056658


Die Korrelation zwischen dem Rabatt und der Retentionrate schwankt je nach Kohorte. Es ist fraglich, ob der Rabatt tatsächlich ein geeigneter Predictor ist.  
Daher wird vorgeschlagen erst die Auswahl der Zeitvariablen zu optimieren und dann am Schluss den Rabatt als Feature mitzugeben und zu schauen, ob es das Modell positiv beeinflusst.

#### Categoric Data

In [79]:
# Erstelle eine Liste, um die Charts zu speichern
charts_cat = []

# Schleife über jede Kategorie in list_category
for cat in list_category:
    chart = alt.Chart(data).mark_bar().encode(
        x=alt.X('mean(retentionrate):Q', title=y_label, sort = 'y'),
        y=alt.Y(f'{cat}', title=cat),
    ).properties(
        title=f'Übersicht Response Variable je {cat}',
        width=300,
        height=300
    ).interactive()
    
    # Füge das Chart der Liste hinzu
    charts_cat.append(chart)

# Kombiniere die Charts in einem Rasterlayout
grid_chart_cat = alt.concat(*charts_cat, columns=3) 

# Zeige das kombinierte Chart an
grid_chart_cat.display()


alt.ConcatChart(...)

- Die Variable `erster_monat_kohorte_fg` scheint relevant für das Modell zu sein. So wird deutlich, dass der erste Monat einer Kohorte nicht die Standardretention ist.
- Ebenfalls wird deutlich, dass beide zeitliche Faktoren (Entstehung der Kohorte und zeitverlauf des Einkaufsverhaltens) in das Modell inkludiert werden sollten.

## Model

#### Prepare category data

In [90]:
model_data = pd.get_dummies(data)
model_data

,erster_monat_kohorte_fg,kohortengroesse_indexiert,identifizierte_kunden_indexiert,rabatt_indexiert,retentionrate,monate_seit_einfuehrung_programm_kohorte_-1,monate_seit_einfuehrung_programm_kohorte_-2,monate_seit_einfuehrung_programm_kohorte_0,monate_seit_einfuehrung_programm_kohorte_1,monate_seit_einfuehrung_programm_kohorte_10,...,monate_seit_existenz_kohorte_33,monate_seit_existenz_kohorte_34,monate_seit_existenz_kohorte_35,monate_seit_existenz_kohorte_36,monate_seit_existenz_kohorte_4,monate_seit_existenz_kohorte_5,monate_seit_existenz_kohorte_6,monate_seit_existenz_kohorte_7,monate_seit_existenz_kohorte_8,monate_seit_existenz_kohorte_9
0,True,0.41,0.41,2.54,100.000000,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,0.41,0.40,7.14,97.560976,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,0.41,0.39,9.28,95.121951,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,0.41,0.38,3.22,92.682927,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,0.41,0.38,7.15,92.682927,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,False,18.36,9.45,17.88,51.470588,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
699,False,18.36,8.40,10.59,45.751634,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
700,True,17.49,17.49,23.76,100.000000,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
701,False,17.49,8.54,11.85,48.827902,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#### Variables

In [93]:
### Gewünschte Spalten
### Es wird im ersten Versuch mal für den Zeitverlauf und die Kohorte jeweils der Monat und das Jahr als feature genommen
cols = ['monat_monat_', 'monat_jahr_', 'kohorte_monat_', 'kohorte_jahr_']
regex_pattern = '|'.join([f'^{col}' for col in cols])
predictor_columns = model_data.filter(regex=regex_pattern).columns.tolist()
predictor_columns

['monat_jahr_2014',
 'monat_jahr_2015',
 'monat_jahr_2016',
 'monat_jahr_2017',
 'monat_monat_1',
 'monat_monat_10',
 'monat_monat_11',
 'monat_monat_12',
 'monat_monat_2',
 'monat_monat_3',
 'monat_monat_4',
 'monat_monat_5',
 'monat_monat_6',
 'monat_monat_7',
 'monat_monat_8',
 'monat_monat_9',
 'kohorte_jahr_2014',
 'kohorte_jahr_2015',
 'kohorte_jahr_2016',
 'kohorte_jahr_2017',
 'kohorte_monat_1',
 'kohorte_monat_10',
 'kohorte_monat_11',
 'kohorte_monat_12',
 'kohorte_monat_2',
 'kohorte_monat_3',
 'kohorte_monat_4',
 'kohorte_monat_5',
 'kohorte_monat_6',
 'kohorte_monat_7',
 'kohorte_monat_8',
 'kohorte_monat_9']

In [138]:
# Pro Spalte wird ein Feature ausgeschlossen, da es sich um kategorische Variablen handelt und das Modell nicht alle bekommen darf
features = [
    # 'monat_jahr_2014',
    'monat_jahr_2015',
    'monat_jahr_2016',
    'monat_jahr_2017',
    # 'monat_monat_1',
    'monat_monat_10',
    'monat_monat_11',
    'monat_monat_12',
    'monat_monat_2',
    'monat_monat_3',
    'monat_monat_4',
    'monat_monat_5',
    'monat_monat_6',
    'monat_monat_7',
    'monat_monat_8',
    'monat_monat_9',
    # 'kohorte_jahr_2014',
    'kohorte_jahr_2015',
    'kohorte_jahr_2016',
    'kohorte_jahr_2017',
    # 'kohorte_monat_1',
    'kohorte_monat_10',
    'kohorte_monat_11',
    'kohorte_monat_12',
    'kohorte_monat_2',
    'kohorte_monat_3',
    'kohorte_monat_4',
    'kohorte_monat_5',
    'kohorte_monat_6',
    'kohorte_monat_7',
    'kohorte_monat_8',
    'kohorte_monat_9',
    'erster_monat_kohorte_fg'
 ]

X = model_data[features]
y = model_data[y_label]

#### Data Splitting

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=42)

### Select model

In [140]:
reg = LinearRegression()

### Training and validation

In [141]:
# cross-validation with 5 folds
scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error') *-1

In [142]:
# store cross-validation scores
df_scores = pd.DataFrame({"lr": scores})

# reset index to match the number of folds
df_scores.index += 1

# print dataframe
df_scores.style.background_gradient(cmap='Blues')

,lr
1,39.915753
2,46.112896
3,37.190890
4,44.733967
5,39.244451


In [143]:
alt.Chart(df_scores.reset_index()).mark_line(
     point=alt.OverlayMarkDef()
).encode(
    x=alt.X("index", bin=False, title="Fold", axis=alt.Axis(tickCount=5)),
    y=alt.Y("lr", aggregate="mean", title="Mean squared error (MSE)")
)

alt.Chart(...)

In [144]:
df_scores.describe().T

,count,mean,std,min,25%,50%,75%,max
lr,5.0,41.439591,3.804106,37.19089,39.244451,39.915753,44.733967,46.112896


### Fit model

In [145]:
# Fit the model to the complete training data
reg.fit(X_train, y_train)

LinearRegression()

In [146]:
# intercept
intercept = pd.DataFrame({
    "Name": ["Intercept"],
    "Coefficient":[reg.intercept_]}
    )

# make a slope table
slope = pd.DataFrame({
    "Name": features,
    "Coefficient": reg.coef_}
)

# combine estimates of intercept and slopes
table = pd.concat([intercept, slope], ignore_index=True, sort=False)

round(table, 3)

,Name,Coefficient
0,Intercept,72.604
1,monat_jahr_2015,-3.547
2,monat_jahr_2016,-4.567
3,monat_jahr_2017,-5.536
4,monat_monat_10,-0.466
5,monat_monat_11,0.874
6,monat_monat_12,1.697
7,monat_monat_2,1.056
8,monat_monat_3,2.022
9,monat_monat_4,0.332


### Evaluation on test set

In [147]:
# obtain predictions
y_pred = reg.predict(X_test)

In [148]:
# R squared
r2_score(y_test, y_pred)

0.8304383039902548

In [149]:
# MSE
mean_squared_error(y_test, y_pred).round(3)

np.float64(36.947)

In [150]:
# MSE
mean_squared_error(y_test, y_pred, squared=False).round(3)

np.float64(6.078)

In [151]:
# MAE
mean_absolute_error(y_test, y_pred).round(3)

np.float64(4.826)

### Save model


?


Save your model in the folder `models/`. Use a meaningful name and a timestamp.

## Conclusions

Der R2 Score erscheint bereits relativ gut. Die Retention Rate kann nur zwischen 0 und 100 liegen, wobei die deskriptive Statistik gezeigt hat, dass die Werte eher zwischen 21 und 61 liegen (Interquartilsabstand). Da erscheint der MAE mit 4,826 noch sehr hoch.